In [1]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import gc
import math

Using TensorFlow backend.


# 1.word_proc_dataframe : 將輸入的資料作斷詞、詞性標註及NER

In [2]:
def word_proc_dataframe(input_Series,
              column_name = 'Text',
              tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)
        column_name(str):要處理的欄位名稱
        tt_seg(tuple):起訖列數
        export_flag(boolean):是否產出檔案
        output_file:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    """
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    """
    
    """
    column_list = list(input_df.columns)
    if column_name in column_list:
        text = input_df[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")
    """
    
    #ls_Test = list(text[tt_begin:tt_end])
    
    ###########      debug        ###############
    print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]),end = tt_seg[1]))
    print("列印前5筆{}".format(input_Series[:5]))
    #print(input_Series[:10])
    #print(input_Series[tt_seg[0]:tt_seg[1]])
    ###########      debug        ###############
    ls_len = len(input_Series)
    

    ls_Test = list(input_Series[:])
    
    
    if (len(ls_Test) > 0):
        print("取得list，長度為：{list_len}\n 載入 WS 資料...".format(list_len = len(ls_Test)))
    else:
        raise Exception("資料長度為0")
        
        
    
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_csv(output_file)
        print("File Exported!!!!!")
    else:
        print("斷詞處理完成，不產生檔案")
        
    #return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list
    return entity_sentence_list


# 2.gen_df:產生df["類別","NER"]供產檔使用

In [ ]:
def gen_df(in_list):
    df_class_list = list()
    df_ner_list = list()
    
    count = 0
    for i in range(len(in_list)):
        for j in range(len(in_list[i])):
            count = count +1
            a = in_list[i].pop()
            df_class_list.append(a[2])
            df_ner_list.append(a[3])
        #print(type(new_list[j]),(new_list[j]))
    #print(count)
    extract_df = pd.DataFrame({'類別':df_class_list,'專有名詞':df_ner_list})
    dd_extract_df = extract_df.drop_duplicates().copy()
    del extract_df
    df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])].copy()
    del dd_extract_df
    df_exp_2 = df_exp[df_exp['專有名詞'].map(len) >= 2].copy()
    del df_exp
    return df_exp_2
    

# 3.proc_seg : 自動處理NER及產生CSV檔

In [12]:
def proc_seg(in_df,seg_len=100):
    #計算輸入資料長度
    df_length = len(in_df)
    #計算要分成幾段處理
    seg_n = math.ceil(df_length/seg_len) 
    
    print("輸入資料長度:{in_len}\n共分成{seg}段".format(in_len = df_length,seg = seg_n))
    
    obj = {}
    for i in range(seg_n):
        #print(i*seg_len,(i+1)*seg_len)
        #print(df_seg_test[(i*100):(i+1)*100])
        obj['df_'+str(i)] = in_df[(i*seg_len):(i+1)*seg_len]   
        
   
    for segment in range(len(obj)):
        
        print("=======================\t開始處理第{}段\t===================\n".format(segment))
        print("第{}段長度：{}".format(('df_'+ str(segment)),len(obj['df_'+ str(segment)])))
        
        ########################################################################################
        # 呼叫word_proc_dataframe，回傳NER list
        ########################################################################################
        
        print("(segment*seg_len:{},(segment+1)*segment):{}".format((segment*seg_len),(segment+1)*seg_len))
        print("=====================================\b呼叫word_proc_dataframe\b=============================\n")
        entity_list = word_proc_dataframe(input_Series = obj['df_'+ str(segment)],
                            tt_seg = ((segment*seg_len),(segment+1)*seg_len),export_flag = False)
        
        print("=====================================呼叫 gen_df=============================\n")
        temp_df = gen_df(entity_list)
        temp_df.sort_values(by=['類別']).to_csv("D:\python_kcc\\NER\\NER_export\\df_out_" + str(segment) + ".csv")
        #print(type(obj['df_'+ str(segment)]))
        print("===========================  第{}段處理完成 ===========================  \n".format(segment))
        
        #print(obj['df_'+ str(segment)])

# 4.讀取檔案及執行自動產檔

In [13]:
#讀檔
full_path = r"D:\python_kcc\NER\剪報系統匯出-正確\all-new(11554).xlsx"
df_full_text = pd.read_excel(full_path)
df_full_text.columns

Index(['序號', '日期', '報別', '內容'], dtype='object')

In [15]:
#選擇要的欄位
df_seg_test = df_full_text["內容"][:50]
# 執行產檔
proc_seg(df_seg_test,12)

# 5 將所有檔案彙整成一個CSV，刪掉重複&案類別排序後匯出

In [19]:
import glob

path = r'D:\python_kcc\NER\NER_export' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [20]:
print(len(frame))
frame.groupby('類別').count()

111878


,專有名詞
類別,
EVENT,4778
GPE,14624
LAW,1842
LOC,7710
ORG,43672
PERSON,39252


In [21]:
frame = frame.drop_duplicates().copy()
print(len(frame))
frame.groupby('類別').count()

37716


,專有名詞
類別,
EVENT,2009
GPE,3994
LAW,720
LOC,2586
ORG,14906
PERSON,13501


In [ ]:
# 匯出檔案
frame.sort_values(by=['類別']).to_csv("D:\python_kcc\\NER\\NER_export\\df_out_all.csv")

In [18]:
# 6.

In [ ]:
frame.des